In [2]:
import numpy as np

# arrays from CPU and GPU runs:
cpu = np.loadtxt("phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv", delimiter=",", skiprows=1)   # or however you stored
gpu = np.loadtxt("gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv", delimiter=",", skiprows=1)

diff = np.abs(cpu - gpu)
max_abs = diff.max(axis=0)   # per column
max_rel = (diff / (np.abs(cpu) + 1e-30)).max(axis=0)

print("max abs per column:", max_abs)
print("max rel per column:", max_rel)

# boolean check to test 1e-12 absolute
print("all abs < 1e-12?", np.all(diff < 1e-12))


max abs per column: [0.00000000e+00 2.50000000e+01 6.48000000e+01 5.00000000e+00
 2.76267187e+00 3.00000000e+02 7.99100000e+03]
max rel per column: [0.00000000e+00 2.50000000e+31 5.98000000e+31 9.30576506e+14
 9.21996721e+15 3.48334734e+15 9.00000000e+30]
all abs < 1e-12? False


In [4]:
import pandas as pd
import numpy as np

cpu = pd.read_csv("phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")
gpu = pd.read_csv("gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")

# Ensure both have identical param columns and same rounding tolerance for keys
# If floating rounding could be an issue, round the parameter columns to reasonable decimals:
decimals = 8
cpu["Ubar_r"] = cpu["Ubar"].round(decimals)
cpu["Mubar_r"] = cpu["Mubar"].round(decimals)
gpu["Ubar_r"] = gpu["Ubar"].round(decimals)
gpu["Mubar_r"] = gpu["Mubar"].round(decimals)

# Merge on rounded params
merged = cpu.merge(gpu, on=["Ubar_r", "Mubar_r"], suffixes=("_cpu","_gpu"))

# Columns to compare (non-parameter)
cols = ["Density", "Phi2", "Energy", "Iters"]
diffs = {}
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    diffs[col] = {"max_abs": d.max(), "max_rel": (d / (np.abs(a) + 1e-30)).max()}

print("Comparison summary (merged on Ubar,Mubar):")
for k,v in diffs.items():
    print(f" {k}: max_abs = {v['max_abs']:.3e}, max_rel = {v['max_rel']:.3e}")

# If you want the rows with largest deviation:
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    idx = np.argmax(d)
    print(f"\nLargest diff in {col}: index {idx}")
    print(merged[["Ubar_r","Mubar_r", f"{col}_cpu", f"{col}_gpu"]].iloc[idx])


Comparison summary (merged on Ubar,Mubar):
 Density: max_abs = 3.803e-05, max_rel = 1.000e+00
 Phi2: max_abs = 8.777e-05, max_rel = 1.000e+00
 Energy: max_abs = 3.083e-09, max_rel = 1.000e+00
 Iters: max_abs = 1.000e+00, max_rel = 1.000e+30

Largest diff in Density: index 9154
Ubar_r         1.400000
Mubar_r       -1.000000
Density_cpu    0.000114
Density_gpu    0.000076
Name: 9154, dtype: float64

Largest diff in Phi2: index 138649
Ubar_r      21.200000
Mubar_r     58.700000
Phi2_cpu     0.001338
Phi2_gpu     0.001250
Name: 138649, dtype: float64

Largest diff in Energy: index 21523
Ubar_r        3.300000e+00
Mubar_r      -1.000000e+00
Energy_cpu    4.647784e-09
Energy_gpu    1.565218e-09
Name: 21523, dtype: float64

Largest diff in Iters: index 0
Ubar_r       0.0
Mubar_r     -5.0
Iters_cpu    8.0
Iters_gpu    9.0
Name: 0, dtype: float64


In [5]:
"2-gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv"

'2-gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv'

In [8]:
import pandas as pd
import numpy as np

cpu = pd.read_csv("phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")
gpu = pd.read_csv("2-gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")

# Ensure both have identical param columns and same rounding tolerance for keys
# If floating rounding could be an issue, round the parameter columns to reasonable decimals:
decimals = 8
cpu["Ubar_r"] = cpu["Ubar"].round(decimals)
cpu["Mubar_r"] = cpu["Mubar"].round(decimals)
gpu["Ubar_r"] = gpu["Ubar"].round(decimals)
gpu["Mubar_r"] = gpu["Mubar"].round(decimals)

# Merge on rounded params
merged = cpu.merge(gpu, on=["Ubar_r", "Mubar_r"], suffixes=("_cpu","_gpu"))

# Columns to compare (non-parameter)
cols = ["Density", "Phi2", "Energy", "Iters"]
diffs = {}
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    diffs[col] = {"max_abs": d.max(), "max_rel": (d / (np.abs(a) + 1e-30)).max()}

print("Comparison summary (merged on Ubar,Mubar):")
for k,v in diffs.items():
    print(f" {k}: max_abs = {v['max_abs']:.3e}, max_rel = {v['max_rel']:.3e}")

# If you want the rows with largest deviation:
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    idx = np.argmax(d)
    print(f"\nLargest diff in {col}: index {idx}")
    print(merged[["Ubar_r","Mubar_r", f"{col}_cpu", f"{col}_gpu"]].iloc[idx])

# Add columnwise mean and RMS difference for a more global check
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    rms = np.sqrt(np.mean(d**2))
    diffs[col].update({"mean_abs": d.mean(), "rms_abs": rms})

print(f" {k}: max_abs = {v['max_abs']:.3e}, mean_abs = {v['mean_abs']:.3e}, rms_abs = {v['rms_abs']:.3e}, max_rel = {v['max_rel']:.3e}")



Comparison summary (merged on Ubar,Mubar):
 Density: max_abs = 3.850e-04, max_rel = 1.000e+00
 Phi2: max_abs = 3.850e-04, max_rel = 1.000e+00
 Energy: max_abs = 1.663e-08, max_rel = 1.000e+00
 Iters: max_abs = 2.000e+04, max_rel = 2.000e+34

Largest diff in Density: index 691
Ubar_r         0.100000
Mubar_r       -1.000000
Density_cpu    0.000646
Density_gpu    0.000261
Name: 691, dtype: float64

Largest diff in Phi2: index 691
Ubar_r      0.100000
Mubar_r    -1.000000
Phi2_cpu    0.000646
Phi2_gpu    0.000261
Name: 691, dtype: float64

Largest diff in Energy: index 691
Ubar_r        1.000000e-01
Mubar_r      -1.000000e+00
Energy_cpu    1.986740e-08
Energy_gpu    3.239876e-09
Name: 691, dtype: float64

Largest diff in Iters: index 40
Ubar_r           0.0
Mubar_r         -1.0
Iters_cpu        0.0
Iters_gpu    20000.0
Name: 40, dtype: float64
 Iters: max_abs = 2.000e+04, mean_abs = 7.296e+01, rms_abs = 7.892e+02, max_rel = 2.000e+34


In [10]:
import pandas as pd
import numpy as np

cpu = pd.read_csv("phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")
gpu = pd.read_csv("3gpu-onsesite-phase_t1.0_U25.0-(0.1)_Mu60.0-(0.1).csv")

# Ensure both have identical param columns and same rounding tolerance for keys
# If floating rounding could be an issue, round the parameter columns to reasonable decimals:
decimals = 8
cpu["Ubar_r"] = cpu["Ubar"].round(decimals)
cpu["Mubar_r"] = cpu["Mubar"].round(decimals)
gpu["Ubar_r"] = gpu["Ubar"].round(decimals)
gpu["Mubar_r"] = gpu["Mubar"].round(decimals)

# Merge on rounded params
merged = cpu.merge(gpu, on=["Ubar_r", "Mubar_r"], suffixes=("_cpu","_gpu"))

# Columns to compare (non-parameter)
cols = ["Density", "Phi2", "Energy", "Iters"]
diffs = {}
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    diffs[col] = {"max_abs": d.max(), "max_rel": (d / (np.abs(a) + 1e-30)).max()}

print("Comparison summary (merged on Ubar,Mubar):")
for k,v in diffs.items():
    print(f" {k}: max_abs = {v['max_abs']:.3e}, max_rel = {v['max_rel']:.3e}")

# If you want the rows with largest deviation:
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    idx = np.argmax(d)
    print(f"\nLargest diff in {col}: index {idx}")
    print(merged[["Ubar_r","Mubar_r", f"{col}_cpu", f"{col}_gpu"]].iloc[idx])

# Add columnwise mean and RMS difference for a more global check
for col in cols:
    a = merged[f"{col}_cpu"].to_numpy()
    b = merged[f"{col}_gpu"].to_numpy()
    d = np.abs(a - b)
    rms = np.sqrt(np.mean(d**2))
    diffs[col].update({"mean_abs": d.mean(), "rms_abs": rms})

print(f" {k}: max_abs = {v['max_abs']:.3e}, mean_abs = {v['mean_abs']:.3e}, rms_abs = {v['rms_abs']:.3e}, max_rel = {v['max_rel']:.3e}")



Comparison summary (merged on Ubar,Mubar):
 Density: max_abs = 3.850e-04, max_rel = 1.000e+00
 Phi2: max_abs = 3.850e-04, max_rel = 1.000e+00
 Energy: max_abs = 1.663e-08, max_rel = 1.000e+00
 Iters: max_abs = 2.000e+04, max_rel = 2.000e+34

Largest diff in Density: index 691
Ubar_r         0.100000
Mubar_r       -1.000000
Density_cpu    0.000646
Density_gpu    0.000261
Name: 691, dtype: float64

Largest diff in Phi2: index 691
Ubar_r      0.100000
Mubar_r    -1.000000
Phi2_cpu    0.000646
Phi2_gpu    0.000261
Name: 691, dtype: float64

Largest diff in Energy: index 691
Ubar_r        1.000000e-01
Mubar_r      -1.000000e+00
Energy_cpu    1.986740e-08
Energy_gpu    3.239876e-09
Name: 691, dtype: float64

Largest diff in Iters: index 40
Ubar_r           0.0
Mubar_r         -1.0
Iters_cpu        0.0
Iters_gpu    20000.0
Name: 40, dtype: float64
 Iters: max_abs = 2.000e+04, mean_abs = 7.296e+01, rms_abs = 7.892e+02, max_rel = 2.000e+34
